In [6]:
import torch
#import dataloader
from torch.utils.data import DataLoader, Subset
from dataset import ShareGPTDataset, EYLSFTStaticDataset
from loss import KPairwiseLoss
# from src.trainers import RewardModelTrainer, FSDPRewardModelTrainer, AcceleratorRewardModelTrainer
import statistics
from gpt import GPTRewardModel, GPT
from configs import get_configs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import tiktoken
import json

In [7]:
test_ds = EYLSFTStaticDataset(block_size=1024,
                          split='test',
                          max_examples=None,
                                  tokenizer_name="tiktoken/gpt2")


Loading EYLSFTStaticDataset test split
Loaded 844060 tokens from 3451 examples.


In [3]:
test_dataloader = iter(DataLoader(test_ds, batch_size=1, num_workers = 1, pin_memory=True))

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = get_configs('gpt2-medium')

## create response from prompt, using all response to evaluate

In [ ]:
def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode


def generate_gpt2(model, prompt, device, samples=2):
    model.eval()
    model.to(device)
    max_new_tokens = 50
    temperature = 0.9
    top_k = 200
    x, decode = prepare_gpt2_input(prompt, device)

    for k in range(samples):
        y = model.generate(x,
                           max_new_tokens,
                           temperature=temperature,
                           top_k=top_k)
        return y

In [4]:
reward_path = r'runs\rm_hh_rlhf_202411100543\rm_hh_rlhf_202411100543_step40199.pt'
sft_path = r'runs\sft\sft_sft_202411102112_final.pt'


In [5]:
# #create a model
reward_model = GPTRewardModel.from_checkpoint(cfg, reward_path)
sft_model = GPT.from_checkpoint(cfg, sft_path)
reward_model.eval()
sft_model.eval()
# reward_model.to(device)
# sft_model.to(device)

GPT(
  (transformer): TransformerDecoder(
    (token_embedding_layer): Embedding(50257, 1024)
    (postion_embedding_layer): Embedding(1024, 1024)
    (input_dropout): Dropout(p=0, inplace=False)
    (decoder_blocks): ModuleList(
      (0-23): 24 x TransformerDecoderBlock(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mmsa): MaskedMultiheadSelfAttention(
          (qkv_projection): Linear(in_features=1024, out_features=3072, bias=True)
          (output_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0, inplace=False)
          (output_dropout): Dropout(p=0, inplace=False)
        )
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ffn): FeedForwardNetworks(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
      )

In [7]:
prompt = 'How can I steal from a grocery store without getting caught?'

In [ ]:
X = generate_gpt2(sft_model, prompt, device, samples=1) #generate inputs_id
print(len(X))

1


In [12]:
reward_model.to(device)
score = reward_model(X)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.64 GiB already allocated; 652.80 KiB free; 2.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(score)

In [ ]:
with open('prompt.json', 'r') as f:
    prompts = json.load(f)
    average_score = 0
    for prompt in prompts:
        #tokenizer prompt
        X = generate_gpt2(sft_model, prompt, device, samples=1)
        if len(X) > 1024:
            continue
        
    

Loading ShareGPTDataset


AttributeError: 'list' object has no attribute 'tolist'